## 2018-2022 Nationwide election results, by precinct

This pulls nationwide precinct level results from 2018, 2020, and 2022 from the
MIT Election Science Lab, from several different files:
- 2018 State: https://doi.org/10.7910/DVN/ZFXEJU
- 2018 US Senate: https://doi.org/10.7910/DVN/DGNAFS
- 2018 US House: https://doi.org/10.7910/DVN/IVIXLK
- 2020 State: https://doi.org/10.7910/DVN/OKL2K1
- 2020 US House: https://doi.org/10.7910/DVN/VLGF2M
- 2020 US Senate: https://doi.org/10.7910/DVN/ER9XTV
- 2020 President: https://doi.org/10.7910/DVN/JXPREB
- 2022 All Levels: https://github.com/MEDSL/2022-elections-official/tree/ad541ea94cb6697c0375a8cffe8e91b60372f50b/individual_states
   not published on dataverse yet, download from github.

I tried to use [State Precinct-Level Returns 2016](https://doi.org/10.7910/DVN/GSZG1O),
but this is ain a different format from the later ones, and can't be parsed by duckdb:
https://github.com/duckdb/duckdb/issues/10929

I tried for a bit using the [programmatic API](https://guides.dataverse.org/en/5.5/api/dataaccess.html#download-by-dataset-api)
to download, but it wasn't working so just gave up.

It also does some additional cleanup of that raw data, eg
- normalize date formats and fips codes.
- normalize "APPROVE" and "FOR" to a simple "YES"
- fill in NULLs when it seems correct to do so

I don't drop any rows or otherwise lose information.
    

In [ ]:
from pathlib import Path

import ibis
from ibis import _

ibis.options.interactive = True

In [ ]:
def parse_date(s):
    # some dates are in the format "mm/dd/yyyy" instead of "yyyy-mm-dd"
    # https://github.com/MEDSL/2022-elections-official/issues/20
    s = s.re_replace(r"(\d\d?)/(\d\d?)/(20)?(\d\d)", r"20\4-\1-\2")
    return s.cast("date")


assert parse_date(ibis.literal("01/02/20")).cast(str).execute() == "2020-01-02"
assert parse_date(ibis.literal("11/8/22")).cast(str).execute() == "2022-11-08"
assert parse_date(ibis.literal("01/02/2020")).cast(str).execute() == "2020-01-02"
assert parse_date(ibis.literal("2020-01-02")).cast(str).execute() == "2020-01-02"


def parse_fips5(s):
    # guard against https://github.com/MEDSL/2022-elections-official/issues/21
    s = s.nullif("NAN")
    s = s.nullif("NA")
    s = s.cast(int).cast(str)
    s = s[:5]
    # pad with 0 if necessary
    s = s.re_replace(r"^(\d{4})$", r"0\1")
    return s


assert parse_fips5(ibis.literal("1234")).execute() == "01234"
assert parse_fips5(ibis.literal("12345")).execute() == "12345"
assert parse_fips5(ibis.literal("12345678")).execute() == "12345"
assert parse_fips5(ibis.literal("NA")).execute() is None
assert parse_fips5(ibis.literal("NAN")).execute() is None
assert parse_fips5(ibis.literal(None, type=str)).execute() is None

In [ ]:
schema = {
    "year": "int16",
    "date": "date",
    "state_po": "string",
    # All of the following are duplicate info from state_po that just take up space.
    # You can look these up pretty easily if you need them.
    # "state": "string",
    # "state_fips": "string",
    # "state_cen": "string",
    # "state_ic": "string",
    "county_name": "string",
    "county_fips": "string",
    "jurisdiction_name": "string",
    "jurisdiction_fips": "string",
    "district": "string",
    "office": "string",
    "magnitude": "int",  # the number seats available for this office
    "special": "boolean",
    "stage": "string",
    "precinct": "string",
    "writein": "boolean",
    "candidate": "string",
    "party_detailed": "string",
    "party_simplified": "string",
    "mode": "string",  # how the votes were cast eg "EARLY", "ABSENTEE", etc
    # can't cast to int because some rows contain "*":
    # https://github.com/MEDSL/2022-elections-official/blob/ad541ea94cb6697c0375a8cffe8e91b60372f50b/README.md
    # Per the Nevada Secretary of State, "*"" in the Votes column indicates low
    # turnout and is hidden to protect voter privacy."
    "votes": "string",
    # "dataverse": "string", # one of {"STATE", "PRESIDENT", "SENATE", "HOUSE", None}
    "readme_check": "boolean",
}


def to_schema(t):
    t = t[schema.keys()]
    t = t.mutate(
        date=parse_date(t.date),
        county_fips=parse_fips5(t.county_fips),
        jurisdiction_fips=parse_fips5(t.jurisdiction_fips),
    )
    t = t.cast(schema)
    return t


def read_raw(p):
    # have to set quote explicitly to get around
    # https://github.com/duckdb/duckdb/issues/11838
    t = ibis.read_csv(p, all_varchar=True, quote='"')
    t = to_schema(t)
    return t

In [ ]:
import zipfile

DATA_DIR = Path("../data")


def unzip_csvs(zip_path: Path, dest: Path) -> Path:
    with zipfile.ZipFile(zip_path, "r") as z:
        # extract any csvs in the zip file
        csvs = [f for f in z.namelist() if f.endswith(".csv")]
        z.extractall(dest, csvs)


def get_github_csvs() -> list[Path]:
    csv_dir = DATA_DIR / "github_2022/csvs"
    if not csv_dir.exists():
        zip_dir = DATA_DIR / "github_2022"
        for zip_path in zip_dir.glob("*.zip"):
            unzip_csvs(zip_path, csv_dir)
    csvs = sorted(csv_dir.glob("*.csv"))
    assert len(csvs) == 51, csvs  # 50 states + DC
    return csvs


def read_csvs():
    paths = [
        *get_github_csvs(),
        *(DATA_DIR / "dataverse_2018_state").glob("*.csv"),
        *(DATA_DIR / "dataverse_2018_us_house").glob("*.csv"),
        *(DATA_DIR / "dataverse_2018_us_senate").glob("*.csv"),
        *(DATA_DIR / "dataverse_2020_state").glob("*.csv"),
        *(DATA_DIR / "dataverse_2020_us_house").glob("*.csv"),
        *(DATA_DIR / "dataverse_2020_us_senate").glob("*.csv"),
        *(DATA_DIR / "dataverse_2020_president").glob("*.csv"),
    ]
    tables = [read_raw(p) for p in paths]
    return ibis.union(*tables)


t = read_csvs()
t = t.distinct()
t = t.cache()
print(t.count())
t

# Fixup Candidates

In [ ]:
t.candidate.topk(100).preview(max_rows=100)

In [ ]:
cand_mapping = {
    "FOR": "YES",
    "AGAINST": "NO",
    "UNDER VOTES": "UNDERVOTES",
    "OVER VOTES": "OVERVOTES",
    "WRITE-IN": "WRITEIN",
    "WRITE-IN VOTES": "WRITEIN",
    "WRITE IN": "WRITEIN",
    "WRITEIN VOTES": "WRITEIN",
    "NONE OF THESE CANDIDATES": "OTHER",
    "OTHER WRITE-INS": "OTHER",
    "BLANK BALLOTS": "BLANK",
    "TIMES BLANK VOTED": "BLANK",
    "BLANK VOTES": "BLANK",
    "BLANKS": "BLANK",
    "BLANK/VOID": "BLANK",
    "SCATTERING": "SCATTERING",
    "SCATTERED VOTES": "SCATTERING",
    "FEDERAL BALLOTS": "FEDERAL",
    "NOT QUALIFIED": "REJECTED",
}
t = t.mutate(candidate=t.candidate.substitute(cand_mapping))

## Fixup Office

In [ ]:
office_mapping = {
    "US PRESIDENT": [
        "US PRESIDENT",
    ],
    "US SENATE": [
        "US SENATE",
    ],
    "US HOUSE": [
        "US HOUSE",
        "REPRESENTATIVE IN CONGRESS",
        "DELEGATE TO THE U.S. HOUSE OF REPRESENTATIVE",
        "DELEGATE TO THE US HOUSE",
        "DELEGATE TO THE HOUSE OF REPRESENTATIVES",
        "REPRESENTATIVE IN CONGRESS",
    ],
    "GOVERNOR": [
        "GOVERNOR",
        "GOVERNOR/LIEUTENANT GOVERNOR",
        "GOVERNOR AND LIEUTENANT GOVERNOR",
    ],
    "LIEUTENANT GOVERNOR": [
        "LIEUTENANT GOVERNOR",
    ],
    "ATTORNEY GENERAL": [
        "ATTORNEY GENERAL",
        "STATE ATTORNEY",
        "STATE ATTORNEY GENERAL",  # 39 States elected an Attorney General in this period
    ],
    "SECRETARY OF STATE": [
        "SECRETARY OF STATE",  # 32 States elected a Secretary of State in this period
    ],
    "STATE TREASURER": ["STATE TREASURER", "TREASURER"],
    "AUDITOR OF STATE": ["STATE AUDITOR", "AUDITOR OF STATE"],
    "STATE SENATE": [
        "STATE SENATE",
        "STATE SENATOR",
    ],
    "STATE HOUSE": [
        "STATE HOUSE",
        "STATE REPRESENTATIVE",
        "HOUSE OF DELEGATES",
    ],
}
office_mapping_flat = {}
for canon, appearances in office_mapping.items():
    for a in appearances:
        office_mapping_flat[a] = canon
t = t.mutate(office=t.office.substitute(office_mapping_flat))

## Fixup District

In [ ]:
is_statewide = _.office.isin(["GOVERNOR", "US SENATE", "US PRESIDENT"])

t.filter(
    is_statewide,
).group_by("office", _.district.fillna("NULL").name("district")).agg(
    n=_.count(),
).order_by(
    _.office,
    _.n.asc(),
).preview(max_rows=40)

In [ ]:
t = t.mutate(district=is_statewide.ifelse("STATEWIDE", _.district))
# also replace "AT-LARGE" with "STATEWIDE"
t = t.mutate(district=_.district.replace("AT-LARGE", "STATEWIDE"))
t.district.topk(20)

In [ ]:
# fixup districts that should be statewide but are NULL
is_dc_congress = (_.state_po == "DC") & (_.office == "US HOUSE")
is_ak_congress = (_.state_po == "AK") & (_.office == "US HOUSE")
is_statewide = is_dc_congress | is_ak_congress
t = t.mutate(district=is_statewide.ifelse("STATEWIDE", _.district))

## Fix Party Detailed

In [ ]:
# "INDEPENDENT",
# "INDEPENDENCE",
# "NONPARTISAN",
# "NO PARTY AFFILIATION",
# "NO AFFILIATION",
# "INDEPENDENT AMERICAN PARTY",
# "INDEPENDENT AMERICAN",
# "NO PARTY",
# "INDEPENDENCE-ALLIANCE",
# "UNENROLLED",
# "INDEPENDENT PARTY",
# "INDEPENDENCE ALLIANCE",
# "NON-PARTISAN DELAWARE",
# "INDEPENDENT NOMINATION",
# "UNAFFILIATED",
# "INDEPENDENT PARTY OF OREGON",
# "NO PARTY PREFERENCE",
# "UNENROLLED INDEPENDENT",
# "COMMON SENSE INDEPENDENT",
# "INDEPENDENT FOR MAINE",
# "A TRUE INDEPENDENT",
party_detailed_mapping = {
    "NO PARTY AFFILIATION": "UNAFFILIATED",
    "NO AFFILIATION": "UNAFFILIATED",
    "NO PARTY": "UNAFFILIATED",
}
t = t.mutate(party_detailed=t.party_detailed.substitute(cand_mapping))

## Fix Mode

In [ ]:
t.mode.topk(30).preview(max_rows=30)

In [ ]:
mode_mapping = {
    "EARLY VOTE": "EARLY",
    "EARLY VOTING": "EARLY",
    "ADVANCED VOTING": "EARLY",
    "ADVANCED": "EARLY",
}
t = t.mutate(mode=_.mode.substitute(mode_mapping))

In [ ]:
t.filter(
    _.mode.isnull(),
).state_po.topk(10)

In [ ]:
def add_n_rows_per_cand(t):
    return (
        t.group_by(
            _.year,
            _.state_po,
            _.office,
            _.district,
            _.county_name,
            _.precinct,
            _.candidate,
        )
        .mutate(
            n_per_candidate=_.count(),
        )
        .order_by(
            _.n_per_candidate.desc(),
        )
    )


add_n_rows_per_cand(t).filter(_.mode.isnull())

In [ ]:
# OK, that looks like all the mode=NULL rows should actually be "TOTAL",
# there is only one row per candidate per precinct. If a candidate
# had more than one row per precinct, then one of them might be TOTAL,
# and the rest would be something else. But that's not the case here.
fixable = ibis.and_(
    _.mode.isnull(),
    _.n_per_candidate == 1,
)
t = (
    add_n_rows_per_cand(t)
    .mutate(mode=fixable.ifelse("TOTAL", _.mode))
    .drop("n_per_candidate")
)
t = t.cache()
t.filter(
    _.mode.isnull(),
).state_po.topk(10)

## Save to file

In [ ]:
# order both so its more user friendly,
# and it might be smaller due to better compression
t = t.order_by(
    _.year,
    _.state_po,
    _.office,
    _.county_name,
    _.district,
    _.precinct,
    _.candidate,
    _.mode,
)
t = t.cache()
t

In [ ]:
def check_integrity(t):
    assert t.year.isin([2018, 2020, 2022]).all().execute()
    assert not (t.district == "NULL").any().execute()


check_integrity(t)

In [ ]:
t.to_parquet(DATA_DIR / "cleaned.parquet")